In [30]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [66]:
training_data = pd.read_csv("trainingSet.csv")
features = [col for col in training_data.columns if col not in ['decision']]

In [42]:
def nbc(t_frac):
    training_data = pd.read_csv("trainingSet.csv")
    features = [col for col in training_data.columns if col not in ['decision']]
    classes = training_data['decision'].unique()
    return nbc_core(features, training_data, 'decision', classes, t_frac)


def nbc_core(feature_cols, data, class_col, class_val, t_frac):
    training_sample = data.sample(frac=t_frac, random_state=47)
    prob_decision = training_sample.groupby([class_col])[class_col].count() / len(training_sample)
    prob_attributes = np.empty((len(class_val), len(feature_cols)), dtype=object)

    for ix, col in enumerate(feature_cols):
        prob = (training_sample.groupby([class_col, col])[class_col].count().unstack(fill_value=0).stack() + 1) / \
                               (training_sample.groupby([class_col])[class_col].count() +
                                len(training_sample[col].sort_values().unique()))

        for i_class in class_val:
            prob_attributes[i_class][ix] = defaultdict(lambda: 1, prob[i_class])

    return prob_decision, prob_attributes


def get_accuracy(data, class_col, classes_val, feature_cols, prob_decision, prob_attributes):
    prob = np.ones((len(data), len(classes_val)))

    # Multiply all class probabilities
    for class_i, class_val in enumerate(classes_val):
        prob[:, class_i] *= prob[:, class_i] * prob_decision[class_i]

    # Multiply all feature probabilities
    for ix, col in enumerate(feature_cols):
        for i_class in classes_val:
            prob[:, i_class] *= data[col].map(prob_attributes[i_class][ix])

    data["prediction"] = np.argmax(prob, axis=1)
    correct_classified = data[data[class_col] == data['prediction']][class_col].count()
    return correct_classified/len(data)

In [43]:
training_data = pd.read_csv("trainingSet.csv")
features = [col for col in training_data.columns if col not in ['decision']]
classes = training_data['decision'].unique()
# Learn probabilities
p_decision, p_attributes = nbc(1)

In [45]:
training_data = pd.read_csv("trainingSet.csv")
features = [col for col in training_data.columns if col not in ['decision']]
classes = training_data['decision'].unique()
# Learn probabilities
p_decision, p_attributes = nbc(1)
# Test probabilities
accuracy_training = get_accuracy(training_data, 'decision', classes, features, p_decision, p_attributes)
print(f"Training Accuracy: {accuracy_training:.2f}")

test_data = pd.read_csv("testSet.csv")
accuracy_test = get_accuracy(test_data, 'decision', classes, features, p_decision, p_attributes)
print(f"Testing Accuracy: {accuracy_test:.2f}")

Training Accuracy: 0.78
Testing Accuracy: 0.75


In [32]:
a = (training_data.groupby(['decision', 'field'])['decision'].count().unstack(fill_value=0).stack() + 1) / \
                               (training_data.groupby(['decision'])['decision'].count() +
                                len(training_data['field'].sort_values().unique()))
p_0 = defaultdict(lambda : 1, a[0])
p_1 = defaultdict(lambda : 1, a[1])
print(p_0[156])
print(p_1[156])
training_data[training_data['field'] == 156] 

1
1


,gender,age,age_o,race,race_o,samerace,importance_same_race,importance_same_religion,field,pref_o_attractive,...,theater,movies,concerts,music,shopping,yoga,interests_correlate,expected_happy_with_sd_people,like,decision


In [7]:
def get_class(x):
    p_estimated_decision = {}
    for val in training_data['decision'].unique():
        p_estimated_decision[val] = p_decision[val]
        for col in features:
            if x[col] in p_attributes[col][val]:
                p_estimated_decision[val] = p_estimated_decision[val] *p_attributes[col][val][x[col]]
    return p_estimated_decision

In [89]:
test_data = pd.read_csv("testSet.csv")
for ix, x in enumerate(features):
    p_0 = p_attributes_0[ix]
    p_1 = p_attributes_1[ix]
p_0

like
(-0.001, 2.0]    0.070473
(2.0, 4.0]       0.213703
(4.0, 6.0]       0.473409
(6.0, 8.0]       0.221207
(8.0, 10.0]      0.021207
dtype: float64

In [109]:
test_data = pd.read_csv("testSet.csv")
accuracy_test = 0
p_0 = np.
ones(len(test_data))
p_1 = np.ones(len(test_data))
for ix, col in enumerate(features):
    p_0 *= test_data[col].map(p_attributes_0[ix])
    p_1 *= test_data[col].map(p_attributes_0[ix])
prediction = 

In [62]:
print("Testing Accuracy:", accuracy_test/len(test_data))

0.7494440326167532
0.7494440326167532


In [63]:
training_data = pd.read_csv("trainingSet.csv")
accuracy_training = 0
for i,x in training_data.iterrows():
    p_estimated_decision = get_class(x)
    accuracy_training += 1 if max(p_estimated_decision, key=p_estimated_decision.get) == x['decision'] else 0

In [64]:
print("Training Accuracy:", accuracy_test/len(test_data))

0.77516218721038
0.77516218721038
